In [13]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr # oh yeah!

In [14]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

In [15]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()
claude = anthropic.Anthropic()

In [4]:
system_message = "You are a helpful assistant"

In [11]:
def message_gpt(prompt):
    messages= [
        { "role": "system", "content": system_message },
        { "role": "user", "content": prompt }
    ]

    completions = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
    )
    return completions.choices[0].message.content

    

In [16]:
message_gpt("What is today's date?")

"I'm sorry, but I don't have real-time capabilities to provide the current date. You can easily find today's date on your device or calendar."

In [17]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [18]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [19]:
def stream_gpt(prompt):
    messages= [
        { "role": "system", "content": system_message },
        { "role": "user", "content": prompt }
    ]

    stream = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        stream=True
    )

    response = ""
    for chunk in stream:
        response+=chunk.choices[0].delta.content or ""
        yield response
    

In [20]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )

    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [26]:
def stream_brochure(company,url,model):
    prompt = f"Generate a company brochure of {company}. Here is their landing page:\n"
    prompt += Website(url).get_contents()

    if(model == "GPT"):
        result=stream_gpt(prompt)
    elif(model == "Claude"):
        result=stream_claude(prompt)
    else:
        raise ValueError("Unknown model")

    yield from result
    

In [27]:
gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
